In [14]:
import ccxt
import pandas as pd
import math
from datetime import datetime, timedelta
# import telegram_bot
import asyncio

#API 키 읽기
with open("../api(JKBOT).txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, #"XTpKrQGSk3GhXzqiEV4OfwGJzmTVcLh8dKGwHo4aQBH4p0mOqPDpIsxdh95tjGVf",
    'secret': secret, #"A0eqZGEWHsyL3NMM6WuDrucIanr7A2YZAnrwXVPhXpf2WGauIANwa5zsoNeNt0hs",
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})

today_date = ""

def ohlcv(symbol, limit=2000):
    btc_ohlcv = binance.fetch_ohlcv(symbol=symbol, timeframe='1d', since=None, limit=limit)
    df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')


    return df
# def cal_amount(usdt_balance, cur_price, units):
#     amount = math.floor((usdt_balance * 1/units) / cur_price) / (1/units)
#     return amount
jongmok = []
markets = binance.load_markets() #마켈로드
balance = binance.fetch_balance() #계좌로드

# 선물종목 담아오기
for market in markets.keys():
    if market.endswith("/USDT"):
        jongmok.append(market[:-5])
        print(jongmok)

# jongmok = ['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH', 'ZEC', 'XTZ', 'BNB', 'ATOM', 'ONT', 'IOTA', 'BAT', 'VET', 'NEO', 'QTUM', 'IOST', 'THETA', 'ALGO', 'ZIL', 'KNC', 'ZRX', 'COMP', 'OMG', 'DOGE', 'SXP', 'KAVA', 'BAND', 'RLC', 'WAVES', 'MKR', 'SNX', 'DOT']


# 거래 최소단위

min_units = {}
for sym in jongmok:
    min_units[sym] = markets[sym+'/USDT']['limits']['market']['min']

#보유종목 조회
def account():
    global account_df
    print("계좌를 조회합니다.")
    balance = binance.fetch_balance()
    positions = balance['info']['positions'] # 포지션 조회
    df = pd.DataFrame(positions, columns = list(positions[0].keys()))
    account_df = df[df['initialMargin'] != "0"]
    position_tot = (pd.to_numeric(account_df['entryPrice']) * abs(pd.to_numeric(account_df['positionAmt']))).sum()
    cur_tot = pd.to_numeric(account_df['initialMargin']).sum()
    # unrealizedProfit = pd.to_numeric(account_df['unrealizedProfit']).sum()
    unrealizedProfit = balance['info']['assets'][6]['unrealizedProfit']
    marginBalance = balance['info']['assets'][6]['marginBalance'].split('.')[0].strip()
    print("보유금액: " +str(marginBalance) + " USDT", "매수금액: " + str(int(cur_tot)) + " USDT", "  미실현손익: " + str(int(round(float(unrealizedProfit)))) +" USDT")
    return account_df

# 현재가 조회
def cur_price(symbol):
    cur_price = binance.fetch_ticker(symbol+'/USDT')['last']
    # print(symbol +"의 현재가는 " + str(cur_price) + "입니다.")
    return cur_price
# 주문
def order(symbol, usdt, side):

    print(symbol + "을 " + str(usdt) + "USD "+ side +"합니다.")
    amount = math.floor((usdt/cur_price(symbol))/min_units[symbol])*min_units[symbol]
    # binance.create_market_order(symbol=symbol+'/USDT', amount=amount, side=side)

# 청산
# 청산
def close_order(symbol):
    print(symbol +"을 모두 청산합니다.")
    balance = binance.fetch_balance()
    positions = balance['info']['positions'] # 포지션 조회
    df = pd.DataFrame(positions, columns = list(positions[0].keys()))
    amount = float(df.loc[df.loc[:, 'symbol'] == symbol.replace("/USDT", "USDT")]['positionAmt'].iloc[0])
    if amount > 0:
        side = 'SELL'
    elif amount <0:
        side = 'BUY'
    binance.create_market_order(symbol=symbol+'/USDT', amount=abs(amount), side=side)

#거래내역 조회
def history():
    history= []
    print("거래내역을 조회합니다.")
    for sym in jongmok:
        d = binance.parse8601()
        fetchTrades = binance.fetch_my_trades(symbol=sym+'/USDT', since=d, limit=4000, params={'order': 'asc'})
        if len(fetchTrades) != 0:
            for k in range(len(fetchTrades)):
                fetchTrades[k]['fee'] = fetchTrades[k]['fee']['cost']
                fetchTrades[k]['realizedPnl'] = fetchTrades[k]['info']['realizedPnl']
                del fetchTrades[k]['info']
                del fetchTrades[k]['fees']
                history.append(fetchTrades[k])

    col = list(history[0].keys())
    col.append('realizedPnl')
    df = pd.DataFrame(history, columns=list(history[0].keys()))
    df['symbol'] = df['symbol'].str.slice(stop=-5)
    df['datetime'] = df['datetime'].str.slice(stop=-5)
    history_df = df.sort_values('datetime', ascending=True)
    history_df['realizedPnl'] = pd.to_numeric(history_df['realizedPnl'])
    history_df = history_df[['datetime', 'order', 'symbol', 'side', 'price', 'amount', 'cost', 'fee', 'realizedPnl']].reset_index(drop=True)
    history_df = history_df.groupby('order', as_index=False).agg({
        'datetime' : 'first',
        'order' : 'first',
        'symbol' : 'first',
        'side' : 'first',
        'price' : 'mean',
        'amount' : 'sum',
        'cost' : 'sum',
        'fee' : 'sum',
        'realizedPnl' : 'sum'
    })
    history_df = history_df.sort_values('datetime', ascending=True).reset_index(drop=True)
    print("조회하였습니다.")
    return history_df

#스캔
def end_order(symbol, amount, side):
    print(symbol + "을 " + str(amount) + " "+ side +"합니다.")
    # binance.create_market_order(symbol=symbol+'/USDT', amount=amount, side=side)

def scan():
    window = 28
    away = 7
    print("지표를 생성중입니다.")
    from tensorflow import keras
    import numpy as np
    model = keras.Sequential()
    model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.load_weights('1d_28_7_(30).h5')
    global signal, ind_dict, ind_df
    x_data = {}
    ind_dict = {}
    ind = []
    ticker = []
    global today_date
    for symbol in jongmok[:40]:
        btc_ohlcv = binance.fetch_ohlcv(symbol=symbol+'/USDT', timeframe='1d', since=None, limit=window+1)
        df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')

        df.set_index('datetime', inplace=True)
        x_data[symbol] = df[['open', 'high', 'low', 'close']][:-1]

        ## 전처리
        mean = x_data[symbol].mean()
        std = x_data[symbol].std()
        x = (x_data[symbol] - mean) / std

        ticker.append(symbol)
        ind.append(model.predict(np.array([x]), verbose=0)[0][0].tolist())

    ind_df = pd.DataFrame([], columns=['symbol', 'ind', 'ascending'])
    ind_df['symbol']=ticker
    ind_df['ind']=ind
    ind_df['ascending'] = abs(ind_df['ind'] - 0.5)+0.5
    ind_df = ind_df.sort_values('ascending', ascending=False)
    print(str(df.iloc[-2].name)[:-9] + "기준으로 지표를 조회하였습니다.")
    today_date = df.index[-2]
    return ind_df

def signal(buy_std, sell_std):
    print("매수조건: " + str(buy_std) + " 매수조건: " + str(sell_std) + " 종목을 검색합니다.")
    global signal_list
    # condition = (ind_df['ind'] > buy_std) | (ind_df['ind'] < sell_std)
    # selected_rows = ind_df[condition]
    # signal_list = selected_rows.values.tolist()
    signal_list = ind_df[:5].values.tolist()
    return signal_list


['BTC']
['BTC', 'ETH']
['BTC', 'ETH', 'BCH']
['BTC', 'ETH', 'BCH', 'XRP']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH', 'ZEC']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH', 'ZEC', 'XTZ']
['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH', 'ZEC', 'XTZ', 'BNB']
['BTC', 'ETH', 'BCH'

In [17]:
window = 28
away = 7
print("지표를 생성중입니다.")
from tensorflow import keras
import numpy as np
model = keras.Sequential()
model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('1d_28_7_(30).h5')
global signal, ind_dict, ind_df
x_data = {}
ind_dict = {}
ind = []
ticker = []
global today_date
for symbol in jongmok[:40]:
        btc_ohlcv = binance.fetch_ohlcv(symbol=symbol+'/USDT', timeframe='1d', since=None, limit=window+1)
        df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')

        df.set_index('datetime', inplace=True)
        x_data[symbol] = df[['open', 'high', 'low', 'close']][:-1]

        ## 전처리
        mean = x_data[symbol].mean()
        std = x_data[symbol].std()
        x = (x_data[symbol] - mean) / std

        ticker.append(symbol)
        ind.append(model.predict(np.array([x]), verbose=0)[0][0].tolist())

지표를 생성중입니다.


In [18]:
btc_ohlcv = binance.fetch_ohlcv(symbol=symbol+'/USDT', timeframe='1d', since=None, limit=window+1)
df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
df.set_index('datetime', inplace=True)

In [18]:
ind

[0.5614363551139832,
 0.6183530688285828,
 0.40103673934936523,
 0.5146623253822327,
 0.43601730465888977,
 0.3463713526725769,
 0.44598284363746643,
 0.532450795173645,
 0.5946106314659119,
 0.7778221964836121,
 0.5464242100715637,
 0.6810256838798523,
 0.6118213534355164,
 0.5684447288513184,
 0.49918630719184875,
 0.5854629874229431,
 0.2952497601509094,
 0.40050628781318665,
 0.34791046380996704,
 0.6191884875297546,
 0.6617311835289001,
 0.6637142300605774,
 0.5157474875450134,
 0.4780842065811157,
 0.36635714769363403,
 0.6570789217948914,
 0.4772312641143799,
 0.36959439516067505,
 0.5846642851829529,
 0.6229511499404907,
 0.5224258303642273,
 0.44519373774528503,
 0.6105321645736694,
 0.332376092672348,
 0.43405869603157043,
 0.471269428730011,
 0.38292863965034485,
 0.17766107618808746,
 0.5410547256469727,
 0.337302565574646]

In [13]:
model.predict(np.array([x]))

1/1 [==============================] - 0s 12ms/step


array([[0.6480135]], dtype=float32)

# 함수 정리
cur_price(symbol) 현재가 조회
order(symbol, usdt, side) 주문
close_order(symbol) 청산
account() 보유계좌
scan() 종목조회
signal(buy_std, sell_std) 매매신호

In [2]:
jongmok

['BTC',
 'ETH',
 'BCH',
 'XRP',
 'EOS',
 'LTC',
 'TRX',
 'ETC',
 'LINK',
 'XLM',
 'ADA',
 'XMR',
 'DASH',
 'ZEC',
 'XTZ',
 'BNB',
 'ATOM',
 'ONT',
 'IOTA',
 'BAT',
 'VET',
 'NEO',
 'QTUM',
 'IOST',
 'THETA',
 'ALGO',
 'ZIL',
 'KNC',
 'ZRX',
 'COMP',
 'OMG',
 'DOGE',
 'SXP',
 'KAVA',
 'BAND',
 'RLC',
 'WAVES',
 'MKR',
 'SNX',
 'DOT',
 'DEFI',
 'YFI',
 'BAL',
 'CRV',
 'TRB',
 'RUNE',
 'SUSHI',
 'SRM',
 'EGLD',
 'SOL',
 'ICX',
 'STORJ',
 'BLZ',
 'UNI',
 'AVAX',
 'FTM',
 'HNT',
 'ENJ',
 'FLM',
 'TOMO',
 'REN',
 'KSM',
 'NEAR',
 'AAVE',
 'FIL',
 'RSR',
 'LRC',
 'MATIC',
 'OCEAN',
 'CVC',
 'BEL',
 'CTK',
 'AXS',
 'ALPHA',
 'ZEN',
 'SKL',
 'GRT',
 '1INCH',
 'CHZ',
 'SAND',
 'ANKR',
 'BTS',
 'LIT',
 'UNFI',
 'REEF',
 'RVN',
 'SFP',
 'XEM',
 'COTI',
 'CHR',
 'MANA',
 'ALICE',
 'HBAR',
 'ONE',
 'LINA',
 'STMX',
 'DENT',
 'CELR',
 'HOT',
 'MTL',
 'OGN',
 'NKN',
 'SC',
 'DGB',
 '1000SHIB',
 'BAKE',
 'GTC',
 'BTCDOM',
 'TLM',
 'IOTX',
 'AUDIO',
 'RAY',
 'C98',
 'MASK',
 'ATA',
 'DYDX',
 '1000XEC',


In [5]:
signal(0.8, 0.2)

매수조건: 0.8 매수조건: 0.2 종목을 검색합니다.


[['REN', 0.910942018032074, 0.910942018032074],
 ['KAVA', 0.09745022654533386, 0.9025497734546661],
 ['XLM', 0.8922787308692932, 0.8922787308692932],
 ['SNX', 0.8915058970451355, 0.8915058970451355],
 ['OMG', 0.887557864189148, 0.887557864189148]]

In [19]:
scan()

지표를 생성중입니다.
2023-03-26기준으로 지표를 조회하였습니다.


,symbol,ind,ascending
37,MKR,0.177661,0.822339
9,XLM,0.777822,0.777822
16,ATOM,0.295250,0.704750
11,XMR,0.681026,0.681026
33,KAVA,0.332376,0.667624
21,NEO,0.663714,0.663714
39,DOT,0.337303,0.662697
20,VET,0.661731,0.661731
25,ALGO,0.657079,0.657079
5,LTC,0.346371,0.653629


In [28]:
ind_df

,symbol,ind,ascending
4,EOS,0.974843,0.974843
33,KAVA,0.967406,0.967406
71,CTK,0.077284,0.922716
60,REN,0.897224,0.897224
43,CRV,0.172196,0.827804
...,...,...,...
7,ETC,0.491330,0.508670
45,RUNE,0.506789,0.506789
38,SNX,0.496598,0.503402
59,TOMO,0.502533,0.502533


In [4]:
ind_df[ind_df['ind'] > 0.5]

,symbol,ind,ascending
59,TOMO,0.957981,0.957981
33,KAVA,0.925256,0.925256
76,GRT,0.864037,0.864037
2,BCH,0.844971,0.844971
4,EOS,0.839745,0.839745
60,REN,0.824056,0.824056
15,BNB,0.807384,0.807384
16,ATOM,0.757815,0.757815
0,BTC,0.735791,0.735791
37,MKR,0.714546,0.714546


In [3]:
signal_list

NameError: name 'signal_list' is not defined

In [40]:
'''청산'''
print("dashboard 파일을 불러옵니다.")
dashboard = pd.read_excel('dashboard.xlsx', index_col='날짜')


condition = (dashboard['만료일'] == today_date)
#
close_sym = dashboard[condition]
print(str(len(close_sym)) + "개의 종목을 청산합니다")
for i in range(len(close_sym)):
    if close_sym['side'][i] == 'SELL':
        side = 'BUY'
    if close_sym['side'][i] == 'BUY':
        side = 'SELL'
    end_order(close_sym['symbol'][i], close_sym['amount'][i], side)
    print(close_sym['symbol'][i] + "종목을" + str(close_sym['amount'][i]) + "개 청산하였습니다.")
# 만료일인 경우 statue를 open에서 close로 변경
dashboard.loc[condition, 'statue'] = 'close'

usdt = 200
scan()

signal(0.65, 0.35)
print("매매를 진행합니다.")
for i in range(len(signal_list)):
    # dashboard = pd.DataFrame('dashboard.xlsx')
    symbol = signal_list[i][0]
    if signal_list[i][1] > 0.5:
        side = 'BUY'
    if signal_list[i][1] < 0.5:
        side = 'SELL'
    amount = math.floor((usdt/cur_price(symbol))/min_units[symbol])*min_units[symbol]
    new_row = pd.DataFrame({
        'symbol' : [symbol], 'ind' : [signal_list[i][1]], 'side' : [side], 'amount' : [amount], 'open_price' : cur_price(symbol), 'statue' : ['open'], '만료일' : [today_date  + timedelta(days=7)]}, columns=dashboard.columns, index=[str(today_date)])
    dashboard = pd.concat([dashboard, new_row], ignore_index=False)
    order(signal_list[i][0], usdt, side)

dashboard['close_price'] = dashboard['symbol'].apply(cur_price)
dashboard.to_excel("(" + str(today_date)[:-9]  +")dashboard.xlsx", index_label='날짜')
dashboard.to_excel('dashboard.xlsx', index_label='날짜')
print("매매를 종료합니다")


dashboard 파일을 불러옵니다.
1개의 종목을 청산합니다
AR을 19.5 SELL합니다.
AR종목을19.5개 청산하였습니다.
지표를 생성중입니다.
2023-03-04기준으로 지표를 조회하였습니다.
매수조건: 0.65 매수조건: 0.35 종목을 검색합니다.
매매를 진행합니다.
ATOM을 200USD BUY합니다.
ALGO을 200USD BUY합니다.
BNB을 200USD SELL합니다.
XTZ을 200USD SELL합니다.
COMP을 200USD BUY합니다.
매매를 종료합니다


In [31]:
close_sym['symbol']

날짜
2023-02-26 00:00:00    AR
Name: symbol, dtype: object

In [22]:
close_sym

,symbol,ind,side,amount,open_price,statue,close_price,만료일
날짜,,,,,,,,


In [33]:
account()

계좌를 조회합니다.
보유금액: 6435 USDT 매수금액: 1963 USDT   미실현손익: -63 USDT


,symbol,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
19,TRXUSDT,200.64743292,1.30420831,-1.29677601,200.64743292,0,1,False,0.069611939614,3.0E7,BOTH,2901,200.64743292,0,1677561815448,0,0
27,DASHUSDT,193.82397081,1.93823970,2.59684756,193.82397081,0,1,False,72.58714648757,5000000.0,BOTH,-2.706,-193.82397081,0,1677766876147,0,0
67,HBARUSDT,401.67920000,4.01679200,-13.56404108,401.67920000,0,1,False,0.0698827406955,5000000.0,BOTH,5942,401.67920000,0,1677766844391,0,0
84,ARUSDT,186.30485406,1.86304854,-13.14114594,186.30485406,0,1,False,10.228,5000000.0,BOTH,19.5,186.30485406,0,1677455500858,0,0
110,ASTRUSDT,183.66389172,3.67327783,-16.35388828,183.66389172,0,1,False,0.08174,5000000.0,BOTH,2447,183.66389172,0,1677455525551,0,0
140,LPTUSDT,178.91840000,1.78918400,-9.30628352,178.91840000,0,1,False,7.352526703499,5000000.0,BOTH,25.6,178.91840000,0,1677766897040,0,0
197,FETUSDT,423.55940778,8.47118815,-7.17438672,423.55940778,0,1,False,0.4491747800587,5000000.0,BOTH,-927,-423.55940778,0,1677767463475,0,0
208,OCEANUSDT,195.08010000,3.90160200,-4.68350000,195.08010000,0,1,False,0.4052,5000000.0,BOTH,493,195.08010000,0,1677455461979,0,0


In [109]:
from binance.client import Client
client = Client(api_key, secret)

In [26]:
account()

계좌를 조회합니다.
보유금액: 6436 USDT 매수금액: 2150 USDT   미실현손익: -66 USDT


,symbol,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
19,TRXUSDT,200.19801000,1.30128706,-1.74619893,200.19801000,0,1,False,0.069611939614,3.0E7,BOTH,2901,200.19801000,0,1677561815448,0,0
27,DASHUSDT,193.01265020,1.93012650,3.40816816,193.01265020,0,1,False,72.58714648757,5000000.0,BOTH,-2.706,-193.01265020,0,1677766876147,0,0
67,HBARUSDT,399.82820758,3.99828207,-15.41503350,399.82820758,0,1,False,0.0698827406955,5000000.0,BOTH,5942,399.82820758,0,1677766844391,0,0
84,ARUSDT,185.68499781,1.85684997,-13.76100219,185.68499781,0,1,False,10.228,5000000.0,BOTH,19.5,185.68499781,0,1677455500858,0,0
110,ASTRUSDT,182.52173000,3.65043460,-17.49605000,182.52173000,0,1,False,0.08174,5000000.0,BOTH,2447,182.52173000,0,1677455525551,0,0
140,LPTUSDT,177.90746060,1.77907460,-10.31722291,177.90746060,0,1,False,7.352526703499,5000000.0,BOTH,25.6,177.90746060,0,1677766897040,0,0
197,FETUSDT,617.48616908,12.34972338,-4.81176916,617.48616908,0,1,False,0.4491747800587,5000000.0,BOTH,-1364,-617.48616908,0,1677766392928,0,0
208,OCEANUSDT,194.24200000,3.88484000,-5.52160000,194.24200000,0,1,False,0.4052,5000000.0,BOTH,493,194.24200000,0,1677455461979,0,0


In [121]:
info = client.futures_account()
info

{'feeTier': 0,
 'canTrade': True,
 'canDeposit': True,
 'canWithdraw': True,
 'updateTime': 0,
 'multiAssetsMargin': False,
 'totalInitialMargin': '1335.49277126',
 'totalMaintMargin': '18.43141473',
 'totalWalletBalance': '6531.78987419',
 'totalUnrealizedProfit': '-58.39230989',
 'totalMarginBalance': '6473.39756430',
 'totalPositionInitialMargin': '1335.49277126',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalCrossWalletBalance': '6531.78987419',
 'totalCrossUnPnl': '-58.39230989',
 'availableBalance': '5137.44724447',
 'maxWithdrawAmount': '5137.44724447',
 'assets': [{'asset': 'DOT',
   'walletBalance': '0.00000000',
   'unrealizedProfit': '0.00000000',
   'marginBalance': '0.00000000',
   'maintMargin': '0.00000000',
   'initialMargin': '0.00000000',
   'positionInitialMargin': '0.00000000',
   'openOrderInitialMargin': '0.00000000',
   'maxWithdrawAmount': '0.00000000',
   'crossWalletBalance': '0.00000000',
   'crossUnPnl': '0.00000000',
   'availableBalance': '0.0000000

In [123]:
info = client.futures_account_trades()
info

[{'symbol': 'NEOUSDT',
  'id': 188895172,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '75.69',
  'realizedPnl': '15.55643884',
  'marginAsset': 'USDT',
  'quoteQty': '971.25408',
  'commission': '0.38850163',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'positionSide': 'BOTH',
  'buyer': True,
  'maker': False},
 {'symbol': 'NEOUSDT',
  'id': 188895173,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '0.59',
  'realizedPnl': '0.12126171',
  'marginAsset': 'USDT',
  'quoteQty': '7.57088',
  'commission': '0.00302835',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'positionSide': 'BOTH',
  'buyer': True,
  'maker': False},
 {'symbol': 'NEOUSDT',
  'id': 188895174,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '0.51',
  'realizedPnl': '0.10481944',
  'marginAsset': 'USDT',
  'quoteQty': '6.54432',
  'commission': '0.00261772',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'position

NameError: name 'self' is not defined